# Analysis of *Felis Catus* behavior

In this notebook we explore the effect of surprises on cats.

# Setup

In [ ]:
devtools::install_github('Gibbsdavidl/CatterPlots')

In [ ]:
library(CatterPlots)
library(tidyverse)

# Data sampling and curation

In the accompanying data table, you can see the data that was meticulously and rigorously collected in the *Felis Catus* behavior experiment. Elapsed time and elevation measurements were gathered from the *Felis Catus* [historical video records](https://www.youtube.com/watch?v=6U_XREUMOAU). 

Each of the samples obtained include the initial point of suprise, max elevation obtained, and final elevation.

Samples with discontinous initial and final elevations (e.g. those of a cat leaping off of a bed or other furniture) were discarded for the purpose of this behavior analysis.

In [ ]:
data <- tibble(
    'sample-name' = c('SC-01','SC-02','SC-03','SC-04','SC-05'),
    'video-ts' = c('0:01:00','0:19:00','0:22:00','0:32:00','1:25:00'),
    't-initial (ms)' =  c(0,0,0,0,0),
    't-max (ms)' = c(249,350,300,305,250),
    't-final (ms)' = c(500,700,620,635,485),
    'max-elevation (mm)' = c(450,750,500,500,350))

# Model creation

We created a mathematical model of surprised *Felis Catus*<sup>*</sup> elevations over time, which closely matches the observed behavior, based upon simplified Newtonian physics.


---



\* a surprised *Felis Catus* is perhaps more accurately named [*Miratus*](https://www.google.com/search?q=latin+Miratus+to+english&rlz=1CAOTWH_enUS896&sxsrf=APq-WBuOE6NGJE-cKAgk7J2wu2FE7w018g%3A1647822636169&ei=LMc3Yuj5BO6c0PEPw_q2wAw&ved=0ahUKEwjov4HX-dX2AhVuDjQIHUO9DcgQ4dUDCA4&uact=5&oq=latin+Miratus+to+english) *Felis Catus*

In [ ]:
options(repr.plot.width=16, repr.plot.height=8)

t <- seq(0, 500, length.out=20) # Units are in milliseconds (ms).
g <- (9.8 * 1000) / (1000^2)   # Gravity (mm/ms**2).
y0 <- 0           # Initial height of cat prior to surprise (mm).
v0y <- 2.45       # Initial vertical velocity of suprised cat (mm/ms).

y <- y0 + v0y * t - 1/2 * g * t^2

purr <- catplot(xs = t, ys = y, cat = 3, size = 0.05, xlim = c(-10,500), ylim = c(-10,310),
                cex.lab = 1.5, cex.axis = 1.5, cex.main = 2.0, cex.sub = 1.5,
                xlab = 'time (ms)', ylab = 'cat elevation (mm)', main = 'Model of surprised cat')

# Application of model to obtained data set

Applying the model allows us to extrapolate intervening data points not recorded during initial data gathering, and to perform repeated theoretical experiments.  This likely would have been quite challenging to do with a real cat, as it is unlikely to have tolerated such shenanigans.

In [ ]:
options(repr.plot.width=16, repr.plot.height=8)

## Append calculated landing velocity to dataframe and some aesthetics.
data <- data %>% mutate(
    'landing-velocity (mm/ms)' = 9.8 / 1000 * (`t-final (ms)` - `t-max (ms)`),
    cat_shape = seq(1, nrow(data)),
    cat_color = c('#33FCFF','#FF0000', '#0495EE','#EE7504', '#000000FF')
)

dim(data)

In [ ]:
extrapolated_data <- pmap_dfr(data, function(`sample-name`, `video-ts`, `t-initial (ms)`, `t-max (ms)`,
                                             `t-final (ms)`, `max-elevation (mm)`, `landing-velocity (mm/ms)`,
                                             cat_shape, cat_color) {
    t <- seq(0, `t-final (ms)`, length.out=20) # Units are in milliseconds (ms).
    g <- (9.8 * 1000) / (1000^2)   # Gravity (mm/ms**2).
    y0 <- 0           # Initial height of cat prior to surprise (mm).
    v0y <- `landing-velocity (mm/ms)`       # Initial vertical velocity of suprised cat (mm/ms).
    y <- y0 + v0y * t - 1/2 * g * t^2
    tibble(
        tt = t,
        yy = y,
        'sample-name' = `sample-name`,
        'video-ts' = `video-ts`,
        't-initial (ms)' =  `t-initial (ms)`,
        't-max (ms)' = `t-max (ms)`,
        't-final (ms)' = `t-final (ms)`,
        'max-elevation (mm)' = `max-elevation (mm)`,
        'landing-velocity (mm/ms)' = `landing-velocity (mm/ms)`,
        cat_shape = cat_shape,
        cat_color = cat_color
    )
})
dim(extrapolated_data)

In [ ]:
purr <- multicat(xs = extrapolated_data$tt, ys = extrapolated_data$yy,
                cat = extrapolated_data$cat_shape, size = 0.05, catcolor = extrapolated_data$cat_color,
                cex.lab = 1.5, cex.axis = 1.5, cex.main = 2.0, cex.sub = 1.5,
                xlab = 'time (ms)', ylab = 'cat elevation (mm)', main = 'Surprised cats')

# Discussion

As you can see, when cats are surprised, the result is an initial rapid gain in elevation followed by a corresponding reduction in elevation. The elevation plateau observed at the temporal mid-point as cat velocity approaches zero, due to the constant deceleration applied to the cat by the force of gravity. It at this mid-point that max elevation is achieved. 

After reaching max elevation and with negligible air-resistance at these velocities, the cat will experience free-fall, aka zero gravity from the cat's frame of reference. Throughout the cat's descent, elevation decreases faster and faster as gravity accelerates them earthbound, until their velocity comes to a stop as they return to their initial starting elevation.

<div class="alert alert-block alert-info">
For more detail on these scientific results, see the relevant <a href='https://github.com/Gibbsdavidl/CatterPlots'>literature</a>. The journal article is expected to appear in Nature on April 1, 2022.
</div>

## Potential future *Felis Catus* modeling

Development of a more complex model could perhaps answer additional questions, such as: 
 * Does air-resistance come into play as velocity increases, and if so, then what sort of an impact does fluffy vs short-haired cat species have?
 * Could terminal velocity (where air resistance cancels out the acceleration due to gravity) be reached on the return trip to earth?
   * What initial veritical leap velocity would be necessary to achieve this?
   * Is it even possible to surprise *Felis Catus* sufficiently for that leap to occur?

# Provenance

In [ ]:
devtools::session_info()

Copyright 2022 The Broad Institute, Inc., Verily Life Sciences, LLC All rights reserved.

This software may be modified and distributed under the terms of the BSD license. See the LICENSE file for details.